# Word2Vec

Существует множество инструментов, которые, обучившись на данных, умеют сопоставлять каждому слову языка некий числовой вектор, который описывает его синтаксические и семантические характеристики. Например, это может быть тематическое моделирование. Мы будем говорить о word2vec.

Overview of embeddings based on factorizations: http://laxxx.org/seminar-20150608-handout.pdf

Rather good overview of riemannian manifold mathematics: http://www.zfm.ethz.ch/~karrasch/Intro_Grassmann.pdf

Major articles:
1. [2013, Efficient Estimation of Word Representations in Vector Space](http://arxiv.org/pdf/1301.3781.pdf)<br>
First basic article about word2vec
2. [2013, Distributed representations of words and phrases and their compositionality](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)<br>
Second basic article about word2vec
3. [2014, word2vec Explained: Deriving Mikolov et al.’s Negative-Sampling Word-Embedding Method](http://arxiv.org/pdf/1402.3722.pdf)<br>
Explanation, what is Skip-Gram model and what is Negative Sampling, which are used in word2vec
4. [2014, Neural Word Embedding as Implicit Matrix Factorization](https://levyomer.files.wordpress.com/2014/09/neural-word-embeddings-as-implicit-matrix-factorization.pdf)<br>
Skip-gram with negative sampling in matrix factorization point of view. Interesting analytical insights, but not interesting experiments
5. [2015, Improving Distributional Similarity with Lessons Learned from Word Embeddings](http://www.openu.ac.il/ISCOL2015/downloads/ISCOL2015_submission25_e_2.pdf)<br>
Not very relevant now
6. [2015, Word Embedding Revisited: A New Representation Learning and Explicit Matrix Factorization Perspective](http://home.ustc.edu.cn/~tianfei/Fei%20Tian's%20Homepage_files/papers/ijcai15.pdf)<br>
Authors provide matrix factorization loss in case of decomposition of the co-occurence matrix. Also they provide simple alternating algorithm for optimization. Interesting, that it outperforms SGNS in case of noisy datasets (minimal word count is low). It is very promising — matrix factorizations give really better performance! However this form of loss function doesn't seem very useful for us — it is easier to approximate the mutual information matrix.
7. [2015, A Primer on Neural Network Models for Natural Language Processing](http://arxiv.org/pdf/1510.00726.pdf)<br>
Overview of neural nets in NLP
8. [2015, Evaluation methods for unsupervised word embeddings](https://www.cs.cmu.edu/~ark/EMNLP-2015/proceedings/EMNLP/pdf/EMNLP036.pdf)<br>
Authors analyse different methods for evaluation of word embeddings. Not very comprehensive overview :(
9. [2015, Model-based Word Embeddings from Decompositions of Count Matrices](http://www1.cs.columbia.edu/~stratos/research/acl15cca.pdf)<br>
Authors try to decompose different matrices, which elements depend on word counters (not only shifted mutual information). Results are pretty comparable to word2vec.
10. [2015, How to Generate a Good Word Embedding?](http://arxiv.org/pdf/1507.05523.pdf)<br>
Rather good explanation, comparison and insights about different embedding methods
11. [2015, Reasoning about Linguistic Regularities in Word Embeddings using Matrix Manifolds](http://arxiv.org/pdf/1507.07636.pdf)<br>
Authors noticed that cosine distance, which is usually used to compare word embeddings, is ad-hod doesn't have to be optimal in terms of analogy tasks. Using riemannian geometry of word points in the embedding space, the authors propose method to build more accurate distance between words.
12. [2015, OUTPERFORMING WORD2VEC ON ANALOGY TASKS WITH RANDOM PROJECTIONS](http://arxiv.org/pdf/1412.6616v1.pdf)<br>
Interesting ad-hoc approach to train model of embeddings (without gradient descent, matrix fatorizations, etc)
13. [2015, A Generative Word Embedding Model and its Low Rank Positive Semidefinite Solution](http://arxiv.org/pdf/1508.03826.pdf)<br>

Как описывается в последних статьях, word2vec можно рассматривать как матричное разложение. В связи с этим есть идея — давайте попробуем использовать римановскую оптимизацию для обучения word2vec.

# Problem description

People mean different models when talk about word2vec. We are talking about Skip-Gram with Negative Sampling model (SGNS) [1,2,3]. The result of this model is a low dimensional vector for each word that describes its syntactic and semantic properties. 

Training of this model is equal [6] to following matrix factorization problem:
$$D \approx C^TW,\quad D\in\mathbb{R}^{|V_c|\times |V_w|}, \quad C \in\mathbb{R}^{d\times |V_c|}, \quad W \in\mathbb{R}^{d\times |V_w|},$$
where $D$ is a training corpus matrix, $C$ is a context matrix and $W$ is a word matrix.

Let us define the following values:
$$
\#(w,c) = D_{wc}, \; \#(w) = \sum_{c \in V_c} \#(w,c), \; \#(c) = \sum_{w \in V_w} \#(w,c), \; |\mathcal{D}| = \sum_{w \in V_w} \sum_{c \in V_c} \#(w,c)
$$

Then the loss function is computed in the following way (see Equation (5) in [4] for details):
$$l(w,c) = \#(w,c)\log\sigma(\vec{w}\cdot\vec{c})+k\frac{\#(w)\#(c)}{|\mathcal{D}|}\log\sigma(-\vec{w}\cdot\vec{c}),$$
where $\vec{c}$ and $\vec{w}$ are the columns of matrices $C$ and $W$ respectively. 

Authors in [4] used MSE measure and SVD algorithm for this low-rank problem, instead of $l(w,c)$ optimization. We can use low-rank optimization methods to search better solutions.

<img src="img/matrices.png">

# Problem statement as explicit matrix factorization (EMF)

**1. Matrix to decompose:**
$$
\mathbf{D} \in \mathbb{R}^{|V_c| \times |V_w|}
$$

**2. Factor matrices:**
$$
\mathbf{D} \approx \mathbf{C^TW}, \; \mathbf{C} \in \mathbb{R}^{|V_c| \times d},\; \mathbf{W} \in \mathbb{R}^{d \times |V_w|}
$$
$\mathbf{w}$ and $\mathbf{c}$ are the columns of matrices $\mathbf{W}$ and $\mathbf{C}$ respectively.

**3. Some supporting values**
$$
\#(w,c) = D_{wc}, \\
\#(w) = \sum_{c \in V_c} \#(w,c), \\
\#(c) = \sum_{w \in V_w} \#(w,c), \\
|\mathcal{D}| = \sum_{w \in V_w} \sum_{c \in V_c} \#(w,c)
$$

**4. Functional:**
$$
\textbf{MF}(\mathbf{D},\mathbf{C^TW}) = \sum_{w \in V_w} \sum_{c \in V_c} \left[ \#(w,c) \log\sigma(\mathbf{c^Tw}) + k\frac{\#(w)\#(c)}{|\mathcal{D}|} \log\sigma(-\mathbf{c^Tw}) \right] \rightarrow \min_{C,W} \\
\textbf{MF}(\mathbf{D},\mathbf{C^TW}) = \sum_{w \in V_w} \sum_{c \in V_c} \left[ d_{wc} \log\sigma(\mathbf{c^Tw}) + b_{wc} \log\sigma(-\mathbf{c^Tw}) \right]
$$

**5. Gradient:**
$$
\nabla\mathbf{MF(C,W)}_{cw} = \left[\frac{\partial\textbf{MF}(\mathbf{D},\mathbf{C^TW})}{\partial \mathbf{C^TW}}\right]_{cw} = \#(w,c)\sigma(\mathbf{-c^Tw}) - k\frac{\#(w)\#(c)}{|\mathcal{D}|} \sigma(\mathbf{c^Tw}) \\
\nabla\mathbf{MF(C,W)}_{cw} = d_{wc} \sigma(\mathbf{-c^Tw}) - b_{wc} \sigma(\mathbf{c^Tw})
$$

here $d_{wc}$ and $b_{wc}$ does not depend on factors, they are only depend on matrix $\mathbf{D}$ and negative sampling parameter $k$

# Methods for solving EMF problem

## 1. Alternating minimization for EMF

Alternating minimization is a simple iterative optimization procedure.

#### Algorithm 1. Alternating minimization for explicit matrix factorization.

**Input:** Co-occurrence matrix $\mathbf{D}$, step-size of gradient descent $\eta$, maximum number of iterations $K$

**Output:** $\mathbf{C}_K$, $\mathbf{W}_K$

**1.** Initialize $\mathbf{C}_i$ and $\mathbf{W}_i$ randomly, $i=1$.

**2.** $\textbf{while}$ $i \leq K$ $\textbf{do}$

**3.** $\;\mathbf{W}_i = \mathbf{W}_{i-1}$

$\;\;\;\; \textbf{repeat}$

$\;\;\;\;\;\;\mathbf{W}_i=\mathbf{W}_i - \mathbf{C}_{i-1} \times \nabla\textbf{MF} {\mathbf{(C_{i-1},W_i)}}$

$\;\;\;\; \textbf{until}$ $\textit{Convergence}$

**4.** $\;\mathbf{C}_i = \mathbf{C}_{i-1}$

$\;\;\;\; \textbf{repeat}$

$\;\;\;\;\;\;\mathbf{C}_i=\mathbf{C}_i - \mathbf{W}_{i} \times \nabla\textbf{MF} {\mathbf{(C_{i},W_i)}}$

$\;\;\;\; \textbf{until}$ $\textit{Convergence}$

**5.** $i = i + 1$

**6.** $\textbf{end while}$

In this algorithm we are minimizing $\mathbf{MF(D,C^TW)}$, so by $\textit{Convergence}$ we mean $\|\mathbf{MF_{old}(D,C^TW)} - \mathbf{MF_{new}(D,C^TW)}\|<\textit{tol}$, where indices **old** and **new** correspond to the previous and the current iteration inside **repeat** respectively and $\textit{tol}$ is some tolerance.

## 2. Riemannian optimization on low-rank manifold

Let $\mathcal{M}_d$ be a manifold of $d$-rank matrices in $\mathbb{R}^{|V_c|\times|V_w|}$, $\mathcal{M}_d = \{X \in \mathbb{R}^{|V_c|\times|V_w|}: \text{rank}\;X=d\}$ and $T_X\mathcal{M}_d$ be a tangent space to this manifold in a point $X$. We want to minimize our functional $\mathbf{MF}(X)$ in $\mathcal{M}_d$ (to find the point $X^*$ such as $X^* = \arg\min_{X \in \mathcal{M}_d} \mathbf{MF}(X)$). This problem is hard as constrained optimization problem, because it is hard to write down $X \in \mathcal{M}_d$ as explicit expression. To solve this problem, we use the following two-step gradient descent procedure.

Let $X \in \mathcal{M}_d$ be a point from our manifold, $X=U \Sigma V^T$ is its SVD and $\nabla\mathbf{MF}(X)$ is a gradient of our functional in the point $X$. Further for notational simplicity we consider $\nabla\mathbf{MF}(X)=F$.

**Step 1 (Projection).** As a gradient vector of our functional $\nabla\mathbf{MF}(X)=F$ only gives a direction in the whole space $\mathbb{R}^{|V_c|\times|V_w|}$, at the first step we project it onto the tangent space $T_X\mathcal{M}_d$.  Defining $P_U=UU^T$, $P_U^{\bot} = I - UU^T$, $P_V = VV^T$ and $P_V^{\bot} = I - VV^T$ we denote the orthogonal projection onto the tangent space at $X$ as

$$
P_{T_X\mathcal{M}_d}: \mathbb{R}^{|V_c|\times|V_w|} \rightarrow T_X\mathcal{M}_d, \\
F \rightarrow \Pi_F = P_U F P_V + P_U^{\bot} F P_V + P_U F P_V^{\bot}.
$$

**Step 2 (Retraction).** As a tangent vector only gives a direction but not the line search itself on the manifold, at the second step a smooth mapping $R_X: T_X\mathcal{M}_d \rightarrow \mathcal{M}_d$ called the retraction is needed to map tangent vectors to the manifold.

$$
R_X: T_X\mathcal{M}_d \rightarrow \mathcal{M}_d, \\
\Pi_F \rightarrow M_F, \\
M_F = \arg\min_{Z \in \mathcal{M}_d} \|X + \Pi_F - Z\|_F.
$$

Optimization algorithm based on proposed method will look like:

#### Algorithm 2. Riemannian optimization for explicit matrix factorization.

**1.** Initialize $X_0 \in \mathcal{M}_d,\; i=1$

**2.** $\textbf{while}$ $i \leq K$ $\textbf{do}$

**3.** $\;\;F_i = \nabla\textbf{MF}(X_{i-1})$

**4.** $\;\;\Pi_i = P_{T_{X_i}\mathcal{M}_d}(F_i)$

**5.** $\;\;M_i = R_{X_i}(\Pi_i)$ 

**6.** $\;\;X_i = X_{i-1} + \eta M_i$ 

**7.** $\textbf{end while}$

## 3. Projector splitting

#### Algorithm 3. Projector splitting for explicit matrix factorization.

**1.** Initialize $X_0 = C_0^T W_0\in \mathcal{M}_d,\; i=1$

**2.** $\textbf{while}$ $i \leq K$ $\textbf{do}$

**3.** $\;\;U_{i-1}, S_{i-1}, V_{i-1}^T = \text{SVD}(X_{i-1})$

**4.** $\;\; C_{i-1} = U_{i-1}\sqrt{S_{i-1}},\;\;W_{i-1} = \sqrt{S_{i-1}}{V_{i-1}}^T$

**5.** $\;\;F_i = \nabla\textbf{MF}(X_{i-1})$

**6.** $\;\;U_i, \_ = \text{QR}((X_{i-1}+F_i)V_{i-1})$

**7.** $\;\;V_i^T, S_{i}^T = \text{QR}((X_{i-1}+F_i)^TU_i)$

**8.** $\;\;X_i = U_i S_i V_i^T$

**9.** $\textbf{end while}$

## 4. Stochastic PS

#### Parameters
* ss -- sample_size (how many words we sample)
* ws -- window_size (size of the window around the word, all words within this window are considered contexts), **default=4**
* k -- negative sampling parameter **default=5**

#### One iteration of algorithm
**1.** Sample $ss$ words $W_s = \{w_1,w_2,\dots,w_{ss}\}$ from the distribution $p(w)$

**2.** For each word $w \in W_s$

$\;\;$ **2a.** Sample $ws$ contexts $C_{pos}=\{c_1,c_2,\dots,c_{ws}\}$ from the distribution $p(c|w)$. For each $c \in C_{pos}$ change the corresponding value of $\nabla\textbf{MF}_{cw}$ by its contribution (encouraging observed pairs): 
$$\nabla\textbf{MF}_{cw} = \nabla\textbf{MF}_{cw} + \sigma(\mathbf{-c^Tw})$$

$\;\;$ **2b.** Sample $k$ contexts $C_{neg}=\{c_1,c_2,\dots,c_{k}\}$ from the unigram distribution on $c$. For each $c_N \in C_{neg}$ change the corresponding value of $\nabla\textbf{MF}_{cw}$ by its contribution (penalizing random pairs):
$$\nabla\textbf{MF}_{cw} = \nabla\textbf{MF}_{cw} - \frac{1}{k} \sum_{c_N \in C_{neg}}\sigma(\mathbf{c_N^Tw})$$

**3.** Move our solution in a direction of obtained stochastic gradient $\nabla\textbf{MF}$.

# Linguistic tasks and datasets

### Word similarity

In word similarity tasks we estimate the measure of closeness of two different words. Datasets for this task contain pairs of words and assesor's score given to them (the more the score is the more similar the words are).

#### Datasets

1) Original wordsim353. 353 pairs of words rated from 0 to 10. Each pair was rated by 13 or 16 assesors and the mean grade was taken. (http://www.cs.technion.ac.il/~gabr/resources/data/wordsim353/)

2) Similarity wordsim353. 203 pairs of words, subset of original wordsim353 which aim to measure similarity of two words.

3) Relatedness wordsim353. 252 pairs of words, subset of original wordsim353 which aim to measure relatedness of two words.

4) MEN. 3000 pairs of words rated from 0 to 50. Each pair was rated by 50 assesors and the sum grade was taken. (http://clic.cimec.unitn.it/~elia.bruni/MEN)

### Analogical reasoning

In analogical reasoning tasks we want to predict missing word from the statement which encounter analogy (e.g. in phrase "London for England is Paris for \_\_\_\_\_" the missing word is most likely "France").

#### Datasets

1) Google Query dataset for analogcial reasoning contains 19544 fourtuples of words, the task is to predict the forth one given the first three words. This dataset cover 14 different analogy relations (e.g. country-capital, country-currency, nationality-adjective etc).